<a href="https://colab.research.google.com/github/sagar4real/CMP7005-PRAC1/blob/main/01_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sagar4real/CMP7005-PRAC1.git

## **Task 1 - DATA HANDLING**





### **1.1 Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math



print("Libraries imported successfully")

### **1.2 Load and Inspect Individual CSV Files**

In [ ]:
# Folder where your CSVs are stored
DATA_FOLDER = "/content/CMP7005-PRAC1/data"

# Pattern to match your files
pattern = os.path.join(DATA_FOLDER, "*_data.csv")

# List all matching files
csv_files = sorted(glob.glob(pattern))

for(file) in csv_files:
    temp = pd.read_csv(file)
    print(file.split('/')[4] , " = " , temp.shape)

### **1.3 Merge All _data.csv Files**

In [ ]:
if len(csv_files) == 0:
    raise FileNotFoundError("No files found !")

# Load and append
df_list = []
for file in csv_files:
    temp = pd.read_csv(file)
    df_list.append(temp)

# Merge all into one DataFrame
merged_df = pd.concat(df_list, ignore_index=True)

### **1.5 Basic Inspection of the Merged Dataset**

In [ ]:
# preview top rows
merged_df.head()

In [ ]:
#preview buttom rows
merged_df.tail()

In [ ]:
# check dataset structure
merged_df.info()

In [ ]:
# check missing values
merged_df.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
# Summary statistics
merged_df.describe(include='all')


**Task 1 Report**

In this exercise I have downloaded the India Air Quality data set from Moodle and there were several csv files named ending in _data.csv. I wanted to use the same data for my analysis therefore I loaded each file into Jupyter Notebook and did an initial evaluation of the data.

First I made a list of all csv files (with the names of the csv files) available in the dataset folder. Then I loaded each one of these files separately. I then concatenated the individual files together into a single data set using **pandas.concat()** with ignore_index=True to prevent duplicate indexes.

Once the data had been merged, I conducted a preliminary assessment of the data using **.head()**, **.info()**, **.isnull().sum()** and **.describe()** to assess the structure of the data, the type of data in each column and how many missing values are present. The merged data set is now stored as **merged_df.csv** and will be the source data for the EDA in Task 2.

## **Task 2 - Exploratory Data Analysis (EDA)**

### **2.1 Fundamental data understanding**

### **2.1.1 Basic shape, columns, dtypes, sample values**

In [ ]:
# 1.1 Shape, columns and dtypes
print("Rows, Columns:", merged_df.shape)
print("\nColumns:")
print(merged_df.columns.tolist())
print("\nData types:")
display(merged_df.dtypes)


### **2.1.2 Count of missing values & percent missing**


In [ ]:
# 1.2 Missing values summary
missing = merged_df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing / len(merged_df) * 100).round(2)
pd.concat([missing, missing_pct.rename("pct")], axis=1).head(30)


### **2.1.3 Unique values for categorical fields & sample city counts**

In [ ]:
# 1.3 Unique categorical summaries
for col in ['City','AQI_Bucket','aqi_bucket','City_Name']:
    if col in merged_df.columns:
        print(f"\nValue counts for {col}:")
        display(merged_df[col].value_counts().head(10))

# generic approach: list object columns and show top values
obj_cols = merged_df.select_dtypes(include=['object']).columns.tolist()
print("\nObject columns detected:", obj_cols)
for c in obj_cols:
    print(f"\nTop values for '{c}':")
    display(merged_df[c].value_counts().head(8))


### **2.2 Data Preprocessing (cleaning, missing handling, duplicates, feature engineering)**

### **2.2.1 Remove exact duplicate rows(if any)**

In [ ]:
def visualize_duplicates(df, title="Duplicate Check", preview_rows=5):
    """
    Visualizes duplicate information in a DataFrame.
    Shows:
      - Count of duplicate rows
      - Percentage of duplicates
      - Preview of duplicated rows
      - Bar chart visualizing duplicates vs uniques
    """
    dup_mask = df.duplicated(keep=False)
    dup_df = df[dup_mask]

    dup_count = dup_df.shape[0]
    total = df.shape[0]
    unique_count = total - dup_count
    percent_dup = round((dup_count / total) * 100, 2)
    percent_unique = round((unique_count / total) * 100, 2)


    # ----- BAR CHART -----
    categories = ['Duplicates', 'Unique']
    counts = [dup_count, unique_count]
    percentages = [percent_dup, percent_unique]
    colors = ['red', 'green']

    fig, ax = plt.subplots(figsize=(8, 6))

    bars = ax.bar(categories, counts, color=colors, edgecolor='black', alpha=0.8)

    # Add count and percentage labels on top of bars
    for i, (bar, count, perc) in enumerate(zip(bars, counts, percentages)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + max(counts)*0.01,
                f'{count} ({perc}%)',
                ha='center', va='bottom', fontweight='bold')

    # Add total count annotation
    # ax.text(0.5, 1.02, f'Total Rows: {total}', transform=ax.transAxes,
    #         ha='center', fontsize=12, fontweight='bold')

    ax.set_ylabel('Number of Rows', fontweight='bold')
    ax.set_title(f'{title}\nDuplicate vs Unique Rows', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    # Set y-axis limit with some padding
    ax.set_ylim(0, max(counts) * 1.15)

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print(f"\n{'='*50}")
    print("DUPLICATE ANALYSIS SUMMARY:")
    print(f"{'='*50}")
    print(f"Total rows: {total}")
    print(f"Duplicate rows: {dup_count} ({percent_dup}%)")
    print(f"Unique rows: {unique_count} ({percent_unique}%)")
    print(f"{'='*50}")

In [ ]:
visualize_duplicates(merged_df)

Upon performing a duplicate row analysis, no duplicate entries were identified in the dataset. Therefore, no duplicate removal was required at this stage.

### **2.2.2 Convert Date to datetime & Extract Month**

In [ ]:
# Convert Date column to datetime (DD/MM/YYYY format)
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format="%d/%m/%Y", errors='coerce')

# Extract the Month for imputation
merged_df['Month'] = merged_df['Date'].dt.month

merged_df[['Date', 'Month']].head()

We convert the Date column into a proper datetime format and extract the Month so we can group the data by City and Month during missing value imputation.

### **2.2.3 Handeling missing values**

In [ ]:
def visualize_missing(df, title="Missing Value Analysis"):
    """
    Visualizes missing values in a DataFrame.
    Shows:
      - Missing count
      - Missing percentage
      - Bar chart of missing percentage
    """

    # Calculate missing data
    missing_count = df.isna().sum()
    missing_percent = (df.isna().sum() / len(df)) * 100

    # Combine into a DataFrame for display
    missing_df = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing %': missing_percent.round(2)
    }).sort_values(by='Missing %', ascending=False)

    print(f"\n===== {title} =====")
    display(missing_df.head(80))   # show top 20 columns with missing values

    # Plot
    plt.figure(figsize=(10,6))
    missing_df['Missing %'].plot(kind='bar', color='salmon')
    plt.title(title)
    plt.ylabel("Percentage Missing (%)")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_missing(merged_df, title="Before Handling Missing Values")


##  Missing Value Handling Method (Explanation)

To handle missing values in the India Air Quality dataset, we use a **three-level median-based imputation strategy**, which is the most suitable and scientifically accurate approach for this type of data.

###  1. City + Month Median (Main Imputation)
Air pollution levels vary significantly between different **cities** and different **months/seasons**.  
Using the *median value for each City–Month* group ensures that:

- Seasonal patterns are preserved  
- City-specific pollution behaviour is maintained  
- Imputation remains realistic and context-aware  

This prevents mixing values from cities with very different pollution levels.

---

###  2. City Median (Backup Imputation)
Some pollutants (e.g., Xylene, Benzene, Toluene, NH3) have long periods where data is not recorded in certain months.

If the City–Month median cannot be computed, we fill missing values using the **City median**.  
This keeps the regional pollution characteristics consistent without relying on unrelated cities.

---

###  3. Global Median (Final Fallback)
If values remain missing after the first two steps, we use the **overall dataset median**.  
This guarantees:
- No missing values remain  
- Imputation stays robust  
- Outliers do not distort the results (median is resistant to extreme values)

---

##  Why This Method Is the Best for This Dataset
This dataset has:
- Very high missingness in VOCs (up to 61%)  
- Large differences between cities  
- Strong seasonal pollution variation  

A simple mean/median imputation would distort pollution trends.

This **three-level hierarchical method** is ideal because it:

✔ Preserves real-world pollution patterns  
✔ Respects city-level differences  
✔ Handles large missingness sensibly  
✔ Avoids introducing bias  
✔ Produces scientifically reliable values  

This prepares the dataset for accurate EDA, modelling, and further analysis.


### **2.2.3.1 Select Numeric Pollutant Columns**

In [ ]:
# List of pollutant + AQI columns we want to impute
pollutant_cols = [
    'PM2.5','PM10','NO','NO2','NOx','NH3',
    'CO','SO2','O3','Benzene','Toluene','Xylene','AQI'
]

# Keep only the ones that actually exist in your dataframe
pollutant_cols = [col for col in pollutant_cols if col in merged_df.columns]

pollutant_cols


We detect the numeric pollutant columns to ensure that only real pollutant values (PM2.5, NO2, SO2, etc.) are cleaned, preventing accidental imputation of columns like City or Date.

### **2.2.3.2 Create the Imputation Function**

In [ ]:
def impute_pollutants(df, pollutant_cols):
    """
    Imputes missing values for pollutant columns using:
    1. City + Month median
    2. City median
    3. Global median
    """
    for col in pollutant_cols:

        # 1. Fill using City + Month median
        df[col] = df.groupby(['City', 'Month'])[col].transform(
            lambda x: x.fillna(x.median())
        )

        # 2. If still missing, fill with City median
        df[col] = df.groupby('City')[col].transform(
            lambda x: x.fillna(x.median())
        )

        # 3. Final fallback: global median
        df[col] = df[col].fillna(df[col].median())

    return df


Missing pollutant values were imputed using a three-level hierarchical median strategy: first at the City–Month level, then at the City level, and finally using the global median as a fallback. This method was chosen because air pollutant concentrations strongly depend on both geographical location and seasonal variation, and this approach preserves these real-world spatial and temporal patterns while ensuring no missing values remain.

In [ ]:
cleaned_mearged_df = impute_pollutants(merged_df, pollutant_cols)

### **2.2.3.3 Extract unique values and counts for AQI_Bucket**

Since **AQI_Bucket** is a categorical variable derived from the numeric AQI values, we first analyse all unique category labels to check for inconsistencies and missing values. We then standardise the categories and reconstruct missing labels using official AQI threshold rules to ensure logical consistency between AQI and AQI_Bucket before further analysis.

In [ ]:
cleaned_mearged_df['AQI_Bucket'] = cleaned_mearged_df['AQI_Bucket'].astype(str).str.strip()

print("Unique AQI_Bucket values:")
print(cleaned_mearged_df['AQI_Bucket'].unique())

print("\nValue counts (including missing as 'nan'):")
cleaned_mearged_df['AQI_Bucket'].value_counts(dropna=False)


In [ ]:
def clean_aqi_bucket(df, aqi_col='AQI', bucket_col='AQI_Bucket'):
    """
    Cleans and fills AQI_Bucket using numeric AQI values.
    Steps:
    1. Standardises text values
    2. Reconstructs missing categories from AQI using official thresholds
    3. Labels remaining missing values as 'Unknown'
    """

    # --- Standardise AQI_Bucket values ---
    df[bucket_col] = df[bucket_col].astype(str).str.strip()
    df[bucket_col] = df[bucket_col].replace(['nan', 'None', 'NaN', ''], np.nan)

    # --- AQI classification rules (India standard) ---
    def classify_aqi_bucket(aqi):
        if pd.isna(aqi):
            return np.nan
        elif aqi <= 50:
            return "Good"
        elif aqi <= 100:
            return "Satisfactory"
        elif aqi <= 200:
            return "Moderate"
        elif aqi <= 300:
            return "Poor"
        elif aqi <= 400:
            return "Very Poor"
        else:
            return "Severe"

    # --- Fill missing AQI_Bucket using AQI ---
    df[bucket_col] = df[bucket_col].fillna(
        df[aqi_col].apply(classify_aqi_bucket)
    )

    # --- Final fallback ---
    df[bucket_col] = df[bucket_col].fillna("Unknown")

    return df


This function standardises the AQI_Bucket column, reconstructs missing category values using the corresponding numeric AQI based on official threshold rules, and assigns ‘Unknown’ where both values are unavailable.”

In [ ]:
removed_missing_value_df = clean_aqi_bucket(cleaned_mearged_df)

In [ ]:
visualize_missing(removed_missing_value_df, title="After Handling Missing Values")

### **2.2.4 Outlier Detection & Treatment**

In this step, we analyse and treat outliers in key pollutant variables such as PM2.5, PM10, NO₂, SO₂, CO, and O₃. Outliers are identified using visual methods (boxplots) and statistical boundaries (IQR/percentile limits), then capped using winsorization to prevent extreme pollution spikes from distorting the analysis and future predictive models.

### **2.2.4.1 Select Key Pollutant Columns for Outlier Analysis**

Outlier detection will be applied only to the primary continuous pollutants (PM2.5, PM10, NO₂, CO, SO₂, and O₃) because these variables exhibit high variability, wide value ranges, and a strong influence on air quality. Categorical variables and sparse VOC measurements were excluded as outlier analysis is not statistically meaningful for them at this stage.

In [ ]:
# Core pollutant columns for outlier analysis
outlier_cols = ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3']

# Keep only columns that actually exist in your dataset
outlier_cols = [col for col in outlier_cols if col in removed_missing_value_df.columns]

outlier_cols


In [ ]:
def visualize_outliers_grid(df, cols, title_prefix="Outlier Visualization"):
    """
    Visualizes outliers using boxplots arranged 2 per row.
    """
    num_cols = len(cols)
    rows = math.ceil(num_cols / 2)

    plt.figure(figsize=(12, 4 * rows))

    for i, col in enumerate(cols, 1):
        plt.subplot(rows, 2, i)
        sns.boxplot(x=df[col], color='skyblue')
        plt.title(f"{title_prefix}: {col}")
        plt.xlabel(col)

    plt.tight_layout()
    plt.show()


Now we use boxplots to visualise outliers because they effectively show the distribution of pollutant values, identify extreme points beyond the whiskers, and allow quick comparison before and after outlier treatment.

In [ ]:
visualize_outliers_grid(removed_missing_value_df, outlier_cols, title_prefix="Before Outlier Treatment")

### **2.2.4.2 IQR Outlier Capping Function for Outlier Treatment**

The Interquartile Range (IQR) method identifies outliers based on the spread of the middle 50% of the data. Values falling below
𝑄
1
−
1.5
×
𝐼
𝑄
𝑅
Q1−1.5×IQR or above
𝑄
3
+
1.5
×
𝐼
𝑄
𝑅
Q3+1.5×IQR are considered extreme. Instead of removing these values, IQR-based capping limits them to the acceptable boundary range, reducing the influence of extreme pollution spikes while preserving all observations.

In [ ]:
def cap_outliers_iqr(df, cols):
    """
    Caps outliers using the IQR method for the given columns.
    Outliers below the lower bound or above the upper bound are clipped.
    """
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df[col] = df[col].clip(lower_bound, upper_bound)

    return df


Outliers were treated using the Interquartile Range (IQR) method, where extreme values were capped at the lower and upper statistical bounds instead of being removed.

In [ ]:
final_draft_df = cap_outliers_iqr(removed_missing_value_df, outlier_cols)


In [ ]:
visualize_outliers_grid(final_draft_df, outlier_cols, title_prefix="After Outlier Treatment")

In [ ]:
removed_missing_value_df.head(20)

### **2.3 Statistcs/computation-based analysis and Visualisation**

### **2.3.1 Statistical Summary(Automatic Numeric Detection)**

In [ ]:
numeric_cols = final_draft_df.select_dtypes(include=['number']).columns

# Generate statistical summary
final_draft_df[numeric_cols].describe().T


An  statistical summary was computed for all numeric variables to quantify central tendency, dispersion, and range using metrics such as mean, standard deviation, quartiles, and extrema. This numerical profiling provides a baseline understanding of pollutant concentration distributions prior to visual exploratory analysis.

### **2.3.2 Univariate Analysis**

In this section, we will perform univariate analysis to understand the distribution and behaviour of individual air-quality variables. Histograms are used to visualise numeric pollutant concentrations because they effectively show how values are spread, whether the data is skewed, and which ranges occur most frequently. A bar chart is used for the categorical variable AQI_Bucket to display the frequency of each air-quality category. These visualisations help identify dominant pollution levels and provide a foundation for deeper analysis in later steps.

In [ ]:
numeric_cols = final_draft_df.select_dtypes(include=['number']).columns.tolist()

exclude_cols = ['Month', 'Year', 'Day']
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

# Limit number of plots if you want (e.g., first 6 numeric columns)
plot_cols = numeric_cols[:6]

# --- Dynamic grid size: 2 columns ---
n_cols = 2
n_rows = math.ceil(len(plot_cols) / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
axes = axes.flatten()

# --- Loop through columns and plot histograms ---
for i, col in enumerate(plot_cols):
    axes[i].hist(final_draft_df[col].dropna(), bins=30)
    axes[i].set_title(f"Distribution of {col}")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Frequency")

# --- Turn off any unused subplots ---
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()



In [ ]:

plt.figure(figsize=(7,4))
final_draft_df['AQI_Bucket'].value_counts().plot(kind='bar')
plt.title("Distribution of AQI Categories")
plt.xlabel("AQI Category")
plt.ylabel("Count")
plt.show()

The histograms show that most pollutants in the dataset are right-skewed, with many low to moderate values and fewer high-concentration days. PM2.5 and PM10 exhibit broad distributions with noticeable peaks, reflecting frequent moderate pollution events. Gaseous pollutants such as NO, NO₂, NOx and NH₃ show heavy right skew, indicating that high concentrations occur occasionally but not regularly. The AQI category distribution shows that ‘Moderate’ and ‘Satisfactory’ air-quality levels dominate the dataset, while ‘Poor’, ‘Very Poor’, and ‘Severe’ categories occur less frequently. Overall, the univariate analysis highlights the variability of pollutant levels and confirms that most pollutants have rare but significant high-value spikes.

### **2.3.3 Bivariate Analysis**

In this section, bivariate analysis is performed to examine the relationships between pairs of pollutants and air-quality indicators. Scatter plots are used to visualise the strength and direction of relationships between variables, while a line plot is used to observe how pollution levels change over time. These visualisations help identify correlations and real-world dependencies between air-quality variables.

In [ ]:
# --- Define bivariate plot pairs dynamically (only if columns exist) ---
bivariate_pairs = []

if 'PM2.5' in final_draft_df.columns and 'AQI' in final_draft_df.columns:
    bivariate_pairs.append(('PM2.5', 'AQI', 'scatter'))

if 'NO2' in final_draft_df.columns and 'O3' in final_draft_df.columns:
    bivariate_pairs.append(('NO2', 'O3', 'scatter'))

if 'Date' in final_draft_df.columns and 'PM2.5' in final_draft_df.columns:
    bivariate_pairs.append(('Date', 'PM2.5', 'line'))

# --- Grid layout (2 plots per row) ---
n_cols = 2
n_rows = math.ceil(len(bivariate_pairs) / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
axes = axes.flatten()

# --- Plot dynamically ---
for i, (x, y, plot_type) in enumerate(bivariate_pairs):

    if plot_type == 'scatter':
        axes[i].scatter(final_draft_df[x], final_draft_df[y], alpha=0.5)
        axes[i].set_title(f"{x} vs {y}")
        axes[i].set_xlabel(x)
        axes[i].set_ylabel(y)

    elif plot_type == 'line':
        axes[i].plot(final_draft_df[x], final_draft_df[y])
        axes[i].set_title(f"{y} Over Time")
        axes[i].set_xlabel("Date")
        axes[i].set_ylabel(y)

# --- Hide unused subplots ---
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


The scatter plot of PM2.5 vs AQI shows a clear positive relationship, indicating that increases in PM2.5 levels strongly contribute to higher AQI values. The NO2 vs O3 scatter shows a weak, scattered relationship, suggesting these pollutants do not vary together consistently and may be influenced by different emission or atmospheric processes. The PM2.5 time-series plot shows strong fluctuations across years, reflecting seasonal variation with higher pollution during winter months and lower levels during monsoon seasons.

### **2.3.4 Multivariate Analysis**

In this section, multivariate analysis is performed to examine the relationships among multiple pollutants simultaneously. A correlation heatmap is used to visualise the strength and direction of relationships between variables, helping to identify strongly related pollutants and potential multicollinearity.

In [ ]:
numeric_cols = final_draft_df.select_dtypes(include=['number'])

# Compute correlation matrix
corr_matrix = numeric_cols.corr()

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f")
plt.title("Correlation Heatmap of Air Quality Variables")
plt.show()


The heatmap shows that PM2.5 and PM10 are strongly correlated (0.69), indicating that fine and coarse particulate matter increase together. AQI is also strongly related to PM2.5 (0.68) and PM10 (0.50), confirming that particulate matter is the main driver of air-quality degradation. NO and NOx show a very strong correlation (0.76), as expected due to their chemical relationship. Other pollutants such as NH3, SO2, O3, and VOCs show weak correlations, indicating they vary more independently.


**Task 2 Report**

The dataset was first explored to understand its structure, data types, and missing-value patterns. Several pollutant variables such as PM2.5, PM10, NH3, and VOCs contained substantial missing data, while City and Date were complete. Duplicate checking confirmed that no repeated records were present. A hierarchical median-based imputation strategy (City–Month → City → Global median) was applied to handle missing numeric values. The categorical variable AQI_Bucket was reconstructed from numeric AQI values using official threshold rules, and any remaining missing labels were assigned as “Unknown.”

Outlier detection was performed on key continuous pollutants (PM2.5, PM10, NO2, CO, SO2, and O3) using boxplots. Extreme values were treated using the Interquartile Range (IQR) capping method to reduce their influence while preserving all observations. Statistical summaries were generated automatically for all numeric features to understand the central tendency, spread, and range of pollutants. Univariate visualisations, including histograms and bar charts, were used to examine individual pollutant distributions and AQI category frequencies.

Bivariate and multivariate analyses were then conducted to explore relationships between variables. Scatter plots showed strong relationships between PM2.5 and AQI, and between nitrogen-based pollutants. A correlation heatmap revealed that PM2.5 and PM10 have the strongest influence on AQI, while several gaseous pollutants and VOCs exhibited weak correlations. Overall, the EDA highlighted key temporal, spatial, and pollutant-level patterns and prepared the dataset for predictive modelling in the next task.

## **Task 3 - Model Building**

In Task 3, we build predictive models to estimate air-quality levels based on pollutant data. This includes selecting an appropriate target variable, preparing machine-learning features, splitting data into training and testing sets, developing baseline and advanced models, and evaluating their performance using metrics such as MAE, RMSE, and R². The goal is to compare models and determine which provides the most accurate and reliable AQI prediction.

### **3.1 Selecting the Target Variable AQI**
Reason:



*   AQI is the most meaningful public-facing air-quality indicator.
*   It directly depends on pollutants.
*   It's numerical → easy to model.










In [ ]:
final_draft_df['AQI'].isna().sum(), final_draft_df['AQI'].describe()

### **3.1 Create X and Y (Features + Target)**

In [ ]:
# Make a working copy
model_df = final_draft_df.copy()

# Columns we should NOT use as numeric features
drop_cols = ['Date', 'AQI_Bucket']

model_df = model_df.drop(columns=drop_cols, errors='ignore')

# Convert City into numeric (one-hot encoding)
model_df = pd.get_dummies(model_df, columns=['City'], drop_first=True)

# Target
y = model_df['AQI']

# Features
X = model_df.drop(columns=['AQI'])


AQI was selected as the target variable since it is complete and provides a meaningful numeric measure of air quality. Categorical variables such as City were converted into dummy variables, and non-predictive fields like Date and AQI_Bucket were removed to create a clean feature matrix for model development.

### **3.2 Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Check shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


The dataset was split into training (80%) and testing (20%) sets using a fixed random seed to ensure reproducible and unbiased evaluation of model performance.”

### **3.4 Train Baseline Model (Linear Regression)**

Linear Regression is used as a baseline to provide a simple reference point for model performance.

In [ ]:
from sklearn.linear_model import LinearRegression

# Initialize model
lr_model = LinearRegression()

# Train model
lr_model.fit(X_train, y_train)


### **3.5 Evalute the Baseline Model**


We evaluate using:

*   **MAE:** Average absolute error
*   **RMSE:** Penalises larger errors
*   **R²:** Proportion of variance explained



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Predictions
y_pred_lr = lr_model.predict(X_test)

# Metrics
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

mae_lr, rmse_lr, r2_lr


### **3.6 Train Random Forest Regressor**

Random Forest can capture non-linear relationships and interactions between pollutants, which Linear Regression cannot. This makes it well-suited for complex environmental data like air quality.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize model
rf_model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

# Train model
rf_model.fit(X_train, y_train)


### **3.7 Evaluate Random Forest Model**

In [ ]:
# Predictions
y_pred_rf = rf_model.predict(X_test)

# Metrics
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

mae_rf, rmse_rf, r2_rf


A Random Forest regression model was trained to capture non-linear relationships between pollutants and AQI. The model achieved lower error values and a higher R² score compared to the baseline Linear Regression model, indicating improved predictive performance.

### **3.7 Comare Models**

In [ ]:
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'MAE': [mae_lr, mae_rf],
    'RMSE': [rmse_lr, rmse_rf],
    'R2 Score': [r2_lr, r2_rf]
})

results


The Random Forest model outperformed Linear Regression across all evaluation metrics, demonstrating its ability to model complex relationships in air-quality data more effectively.”

### **3.7.1 Bar Chart comparisons of Models**

In [ ]:
metrics = ['MAE', 'RMSE', 'R2 Score']
x = np.arange(len(metrics))
width = 0.35

plt.figure(figsize=(8,5))

plt.bar(x - width/2, results.loc[0, metrics], width, label='Linear Regression')
plt.bar(x + width/2, results.loc[1, metrics], width, label='Random Forest')

plt.xticks(x, metrics)
plt.ylabel('Metric Value')
plt.title('Model Performance Comparison')
plt.legend()
plt.show()


The bar chart shows that the Random Forest model achieves lower MAE and RMSE and a higher R² score compared to Linear Regression, indicating superior predictive accuracy and better generalisation to unseen data.

### **3.7.2 Actual vs Predicted Scatter Plot**

In [ ]:
plt.figure(figsize=(12,5))

# Linear Regression
plt.subplot(1,2,1)
plt.scatter(y_test, y_pred_lr, alpha=0.4)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()])
plt.title('Linear Regression: Actual vs Predicted')
plt.xlabel('Actual AQI')
plt.ylabel('Predicted AQI')

# Random Forest
plt.subplot(1,2,2)
plt.scatter(y_test, y_pred_rf, alpha=0.4)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()])
plt.title('Random Forest: Actual vs Predicted')
plt.xlabel('Actual AQI')
plt.ylabel('Predicted AQI')

plt.tight_layout()
plt.show()


The Random Forest model shows predictions more closely aligned with the diagonal reference line, indicating better accuracy and reduced prediction error compared to Linear Regression.

Both numeric metrics and visual comparisons confirm that the Random Forest model outperforms the Linear Regression baseline, making it the preferred model for AQI prediction in this study.

**Task 3 Report**

In this task, predictive models were developed to estimate Air Quality Index (AQI) using the cleaned and preprocessed air-quality dataset. AQI was selected as the target variable because it provides a comprehensive numeric measure of air quality and contained no missing values. The dataset was prepared by removing non-predictive fields such as Date and AQI_Bucket, and categorical variables such as City were converted into numerical form using one-hot encoding. The data was then split into training (80%) and testing (20%) sets to enable unbiased evaluation of model performance.

A baseline Linear Regression model was first trained to establish a reference level of performance. This model assumes a linear relationship between pollutant concentrations and AQI. Model performance was evaluated using Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and the R² score. To improve predictive accuracy, a Random Forest regression model was then developed. Random Forest was chosen due to its ability to capture non-linear relationships and complex interactions among pollutants. The Random Forest model achieved lower error values and a higher R² score than the Linear Regression model, indicating superior predictive capability.

Model comparison was further supported using visualisations, including bar charts of evaluation metrics and Actual vs Predicted scatter plots. These visual analyses showed that Random Forest predictions aligned more closely with actual AQI values, while Linear Regression exhibited greater dispersion and error. Overall, the Random Forest model outperformed the baseline model and was identified as the most suitable approach for AQI prediction in this study, demonstrating the effectiveness of ensemble methods for modelling complex environmental data.

## **Task 4 - Application Development (Streamlit App)**

This task demonstrates the practical deployment of the developed AQI prediction model through an interactive application, allowing users to explore air-quality data and generate AQI predictions dynamically.

### **4.1 Save the Trained Random Forest Model**

In [ ]:
import joblib

# Create models folder
os.makedirs("models", exist_ok=True)

# Save trained model
joblib.dump(rf_model, "/CMP7005-PRAC1/models/aqi_random_forest_model.pkl")

# Save feature names
joblib.dump(X.columns.tolist(), "/CMP7005-PRAC1/models/feature_names.pkl")

print("Model and feature names saved successfully.")



In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

# Load model and feature names
model = joblib.load("models/aqi_random_forest_model.pkl")
feature_names = joblib.load("models/feature_names.pkl")

st.title("AQI Prediction Application")
st.write("Predict Air Quality Index using pollutant values")

# Sidebar inputs
st.sidebar.header("Enter Pollutant Values")

PM25 = st.sidebar.number_input("PM2.5", 0.0, 1000.0, 50.0)
PM10 = st.sidebar.number_input("PM10", 0.0, 1000.0, 100.0)
NO2  = st.sidebar.number_input("NO2", 0.0, 500.0, 40.0)
CO   = st.sidebar.number_input("CO", 0.0, 50.0, 1.0)
SO2  = st.sidebar.number_input("SO2", 0.0, 500.0, 10.0)
O3   = st.sidebar.number_input("O3", 0.0, 500.0, 30.0)

# Create aligned input
input_df = pd.DataFrame(0, index=[0], columns=feature_names)
input_df['PM2.5'] = PM25
input_df['PM10'] = PM10
input_df['NO2'] = NO2
input_df['CO'] = CO
input_df['SO2'] = SO2
input_df['O3'] = O3

if st.button("Predict AQI"):
    prediction = model.predict(input_df)[0]
    st.success(f"Predicted AQI: {int(prediction)}")
